#인구소멸 위기지역 - 전처리 (Part 1)

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [5]:
data_folder = '/content/mnt/MyDrive/Data/인구소멸위기지역'

In [9]:
!ls -l $data_folder

total 656
-rw------- 1 root root 134740 Sep  6 00:45  101_DT_1B040M5_Y_2020.csv
-rw------- 1 root root 142202 Sep  6 00:51  시군구_성_연령_5세_별_인구2_2020.csv
-rw------- 1 root root  14355 Sep  5 02:13 'draw_korea_raw(2021).xlsx'
-rw------- 1 root root   5269 Sep  5 02:41  draw_map.py
-rw------- 1 root root 373148 Dec  2  2020  skorea_municipalities_geo_simple.json


In [15]:
df = pd.read_csv(f'{data_folder}/시군구_성_연령_5세_별_인구2_2020.csv', encoding='euc-kr')
print(df.shape)
df.head(266).tail()

(837, 28)


,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
261,경기도,수원시,권선구,계,367862,15312,18262,16317,17781,24929,29755,26933,32283,29956,31219,31627,30179,23650,14267,9905,7117,8375,4844,3531,2431,880,189,32
262,경기도,수원시,권선구,남자,185541,7933,9354,8410,9327,12825,15670,14053,16856,15681,15862,15228,15149,11773,6941,4684,3069,2728,1770,958,719,201,33,6
263,경기도,수원시,권선구,여자,182321,7379,8908,7907,8454,12104,14085,12880,15427,14275,15357,16399,15030,11877,7326,5221,4048,5647,3074,2573,1712,680,156,27
264,경기도,수원시,팔달구,계,172567,4379,5577,6198,8072,12530,15887,12316,12320,12370,14785,15060,14905,12648,8656,6474,5067,5327,3200,2127,1467,514,132,14
265,경기도,수원시,팔달구,남자,86722,2181,2853,3196,4119,6273,8549,6723,6514,6360,7598,7594,7516,6226,4088,2956,2161,1818,1203,615,464,119,31,2


In [16]:
df['시군구'][df.행정구.notnull()] = df.행정구[df.행정구.notnull()]
df.head(266).tail()

,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
261,경기도,권선구,권선구,계,367862,15312,18262,16317,17781,24929,29755,26933,32283,29956,31219,31627,30179,23650,14267,9905,7117,8375,4844,3531,2431,880,189,32
262,경기도,권선구,권선구,남자,185541,7933,9354,8410,9327,12825,15670,14053,16856,15681,15862,15228,15149,11773,6941,4684,3069,2728,1770,958,719,201,33,6
263,경기도,권선구,권선구,여자,182321,7379,8908,7907,8454,12104,14085,12880,15427,14275,15357,16399,15030,11877,7326,5221,4048,5647,3074,2573,1712,680,156,27
264,경기도,팔달구,팔달구,계,172567,4379,5577,6198,8072,12530,15887,12316,12320,12370,14785,15060,14905,12648,8656,6474,5067,5327,3200,2127,1467,514,132,14
265,경기도,팔달구,팔달구,남자,86722,2181,2853,3196,4119,6273,8549,6723,6514,6360,7598,7594,7516,6226,4088,2956,2161,1818,1203,615,464,119,31,2


In [17]:
# 소계 데이터 제외
df=df[df.시군구 != '소계']
print(df.shape)
df.head()

(750, 28)


,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
6,서울특별시,종로구,NaN,계,144866,3309,4549,5120,6182,10602,12668,9995,9796,9650,12221,12287,12160,10735,7637,6245,5450,6265,3682,2583,1749,639,169,26
7,서울특별시,종로구,NaN,남자,70613,1668,2297,2575,3099,4976,6526,5005,4820,4636,5993,6072,6180,5409,3744,2842,2369,2406,1517,889,642,189,52,6
8,서울특별시,종로구,NaN,여자,74253,1641,2252,2545,3083,5627,6142,4991,4976,5014,6228,6215,5981,5326,3893,3403,3081,3859,2165,1694,1107,451,117,20
9,서울특별시,중구,NaN,계,121520,3483,3611,3093,3869,7605,10768,10031,10051,8465,9184,9864,10108,9512,6872,5475,4501,5030,2904,2126,1429,558,124,16
10,서울특별시,중구,NaN,남자,59536,1772,1852,1550,1976,3543,5289,4981,5051,4376,4636,5005,5127,4782,3380,2523,1943,1754,1125,629,439,153,32,6


In [18]:
df['20~39세'] = df['20 - 24세'] + df['25 - 29세'] + df['30 - 34세'] + df['35 - 39세']
df['65세 이상'] = df['65 - 69세'] + df['70 - 74세'] + df['75 - 79세'] + df['80 - 84세'] + df['85세 이상']
df = df[['광역시도', '시군구', '구분', '인구수', '20~39세', '65세 이상']]
df.head()

,광역시도,시군구,구분,인구수,20~39세,65세 이상
6,서울특별시,종로구,계,144866,43061,25597
7,서울특별시,종로구,남자,70613,21327,11361
8,서울특별시,종로구,여자,74253,21736,14236
9,서울특별시,중구,계,121520,38455,21878
10,서울특별시,중구,남자,59536,18864,9600


In [21]:
# pivot table을 만들어서 시군구 단위로 grouping
pop = pd.pivot_table(df,
                     index = ['광역시도', '시군구'],
                     columns = ['구분'],
                     values = ['인구수', '20~39세','65세 이상'])
print(pop.shape)
pop.head()

(250, 9)


20~39세               65세 이상                   인구수                
구분             계     남자     여자      계     남자     여자       계      남자      여자
광역시도 시군구                                                                   
강원도   강릉시  47006  25199  21807  43830  18604  25227  211643  105025  106618
      고성군   5217   3175   2042   7755   3237   4519   26792   13864   12928
      동해시  18727  10469   8258  17573   7624   9950   89814   45572   44242
      삼척시  13583   7469   6117  16077   6766   9312   65623   33275   32348
      속초시  18080   9909   8171  15170   6345   8826   81497   40312   41185

In [22]:
# 인구 소멸 비율 계산
pop['소멸비율'] = pop['20~39세','여자'] / pop['65세 이상', '계']        # 2중 인덱스 불러오는 방법
pop.head()

20~39세               65세 이상  ...     인구수                      소멸비율
구분             계     남자     여자      계  ...       계      남자      여자          
광역시도 시군구                               ...                                  
강원도   강릉시  47006  25199  21807  43830  ...  211643  105025  106618  0.497536
      고성군   5217   3175   2042   7755  ...   26792   13864   12928  0.263314
      동해시  18727  10469   8258  17573  ...   89814   45572   44242  0.469925
      삼척시  13583   7469   6117  16077  ...   65623   33275   32348  0.380481
      속초시  18080   9909   8171  15170  ...   81497   40312   41185  0.538629

[5 rows x 10 columns]

In [23]:
pop['소멸위기지역'] = pop.소멸비율 < 0.5
pop['소멸위기고위험지역'] = pop.소멸비율 < 0.2
pop.head()

20~39세               65세 이상  ...     인구수      소멸비율 소멸위기지역 소멸위기고위험지역
구분             계     남자     여자      계  ...      여자                           
광역시도 시군구                               ...                                   
강원도   강릉시  47006  25199  21807  43830  ...  106618  0.497536   True     False
      고성군   5217   3175   2042   7755  ...   12928  0.263314   True     False
      동해시  18727  10469   8258  17573  ...   44242  0.469925   True     False
      삼척시  13583   7469   6117  16077  ...   32348  0.380481   True     False
      속초시  18080   9909   8171  15170  ...   41185  0.538629  False     False

[5 rows x 12 columns]

In [25]:
# 인구소멸 위기지역
crisis_region = pop[pop.소멸위기지역].index.get_level_values(1)     # 두번째 인덱스를 가져와라 (광역시도 = 0, 시군구 = 1)
print(len(crisis_region))
print(crisis_region)

102
Index([' 강릉시', ' 고성군', ' 동해시', ' 삼척시', ' 양구군', ' 양양군', ' 영월군', ' 인제군', ' 정선군',
       ' 철원군',
       ...
       ' 청양군', ' 태안군', ' 홍성군', ' 괴산군', ' 단양군', ' 보은군', ' 영동군', ' 옥천군', ' 음성군',
       ' 제천시'],
      dtype='object', name='시군구', length=102)


In [26]:
# 인구소멸위기 고위험지역
high_crisis_region = pop[pop.소멸위기고위험지역].index.get_level_values(1)
print(len(high_crisis_region))
print(high_crisis_region)

22
Index([' 남해군', ' 산청군', ' 의령군', ' 하동군', ' 합천군', ' 군위군', ' 봉화군', ' 영덕군', ' 영양군',
       ' 의성군', ' 청도군', ' 청송군', ' 고흥군', ' 곡성군', ' 보성군', ' 신안군', ' 함평군', ' 임실군',
       ' 부여군', ' 서천군', ' 청양군', ' 보은군'],
      dtype='object', name='시군구')


In [27]:
pop.columns.get_level_values(0)

Index(['20~39세', '20~39세', '20~39세', '65세 이상', '65세 이상', '65세 이상', '인구수',
       '인구수', '인구수', '소멸비율', '소멸위기지역', '소멸위기고위험지역'],
      dtype='object')

In [28]:
for i in range(len(pop.columns.get_level_values(0))):
    print(pop.columns.get_level_values(0)[i]+pop.columns.get_level_values(1)[i])

20~39세계
20~39세남자
20~39세여자
65세 이상계
65세 이상남자
65세 이상여자
인구수계
인구수남자
인구수여자
소멸비율
소멸위기지역
소멸위기고위험지역


In [29]:
tmp_col = [pop.columns.get_level_values(0)[i] + \
           pop.columns.get_level_values(1)[i]
           for i in range(len(pop.columns.get_level_values(0)))]
pop.columns = tmp_col
pop.reset_index(inplace=True)
pop['시군구'] = pop.시군구.apply(lambda x: x.strip())
pop.head()

,광역시도,시군구,20~39세계,20~39세남자,20~39세여자,65세 이상계,65세 이상남자,65세 이상여자,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역
0,강원도,강릉시,47006,25199,21807,43830,18604,25227,211643,105025,106618,0.497536,True,False
1,강원도,고성군,5217,3175,2042,7755,3237,4519,26792,13864,12928,0.263314,True,False
2,강원도,동해시,18727,10469,8258,17573,7624,9950,89814,45572,44242,0.469925,True,False
3,강원도,삼척시,13583,7469,6117,16077,6766,9312,65623,33275,32348,0.380481,True,False
4,강원도,속초시,18080,9909,8171,15170,6345,8826,81497,40312,41185,0.538629,False,False


In [30]:
pop.to_csv(f'{data_folder}/시군구_전처리완료.csv', encoding='euc-kr', index=False)